In [1]:
import sqlite3
from xmlrpc.server import SimpleXMLRPCServer

# Создаем сервер статистики
stats_server = SimpleXMLRPCServer(('localhost', 7000))

# Подключаемся к базе данных SQLite
conn = sqlite3.connect('log.db')
c = conn.cursor()

# Создаем таблицу, если она не существует
c.execute('''CREATE TABLE IF NOT EXISTS events
             (id INTEGER PRIMARY KEY AUTOINCREMENT,
             types TEXT, event_name TEXT, event_data TEXT)''')

# Определяем функцию для сохранения событий в базе данных
def save_event(types, event_name, event_data):
    # Вставляем данные события в таблицу
    c.execute("INSERT INTO events (types, event_name, event_data) VALUES (?, ?, ?)",
              (types, event_name, event_data))
    # Сохраняем изменения
    conn.commit()
    return "Событие успешно сохранено"
# Регистрируем функцию сохранения события на сервере статистики
stats_server.register_function(save_event)

def select_event(types, end_time, start_time):
    # выбираем данные из таблицы по промежутку
    result = c.execute("SELECT * FROM events WHERE types == ? AND event_data>=? AND event_data<=?",
              (types, start_time, end_time))
    # Сохраняем изменения
    conn.commit()
    return str(result)
# Регистрируем функцию сохранения события на сервере статистики
stats_server.register_function(select_event)




print("Listening on port 7000...")

# Запускаем сервер статистики
stats_server.serve_forever()




Listening on port 7000...


127.0.0.1 - - [27/Dec/2023 18:39:14] "POST /RPC2 HTTP/1.1" 200 -


KeyboardInterrupt: 